# *FB buddy*
- a buddy sorrend kinyerése a profilod html-jéből
- a sorend tárolása google drive-odon sqlite3 adatbázisban (először létrehozza, ehhez engedélyezned kell a hozzáférést)
- ha korábbi file-t használsz, akkor a dátumot át kell állítani. Alalpértelmezetten az aktuális dátumot használja.
- az utolsó 50 rekord (azaz amit betöltött) megmutatása

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from datetime import datetime    
import re
import sqlite3
from ipywidgets import widgets
from IPython.display import display

conn = sqlite3.connect(r'/content/drive/MyDrive/fb_buddy_v1.db')

main_display = widgets.Output()

def tablairas(inputs):
    input_file = list(upload.value.values())[0]
    content = input_file['content'].decode('utf-8')
    lista =re.findall('buddy_id(.+?)picture', content) 
    for i in range(len(lista)):
        lista[i] = re.findall('name\"\:"(.+?)\"\,', lista[i])
    df = pd.DataFrame(lista, columns={'name'})
    dict_hu = {'u00c1': 'Á','u00e1': 'á','u00f3': 'ó','u00e9': 'é','u0151': 'ő','u00f6': 'ö','u00ed': 'í','u00c9': 'É','u00fa': 'ú','u00fc': 'ü'}
    for key in dict_hu.keys():
        df['name'] = df['name'].str.replace(key, dict_hu[key])
        df['name'] = df['name'].str.replace('\\', '', regex=True)
    df['rn']=df.index+1
    df['date']=datum.value.strftime('%Y.%m.%d')
    df.columns =['name', 'rn','date']
    df.to_sql('fb_buddy', conn, if_exists='append', index=False)
    lbl1.value = 'Betöltve'



upload = widgets.FileUpload(
    accept='.html',
    multiple=False
)

datum = widgets.DatePicker(
    description='Dátum választás',
    disabled=False,
    value = datetime.today()
)

lbl1 = widgets.Label('')

display(datum, upload, lbl1)
upload.observe(tablairas, names='value')

In [3]:
conn.commit()

In [ ]:
query = """SELECT distinct * FROM fb_buddy"""
df = pd.read_sql(query, conn)
df.tail(50)

# *Időbeli változás*

In [5]:
df_ido = pd.read_sql("select distinct date from fb_buddy", con=conn)
df.sort_values(by=['date'], inplace=True)
df_ido.sort_values(by=['date'], inplace=True)


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

#df = result

@interact
def scatter_plot(Név=list(sorted(df['name'].unique()))):
        df1 = df[df['name']==Név]
        result = pd.merge(df_ido,
                 df1[['date', 'name', 'rn']],
                 on='date', 
                 how='left')
        df1=result
        df1[["date","rn"]].plot(kind='line', x='date', y='rn')
        

# *Heatmap*

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_sql("select distinct * from fb_buddy", con=conn)
var1 = df.groupby(['name'])['rn'].agg('count').reset_index(name='counts').sort_values(by=['counts','name'], ascending=False)['counts'].unique()[1] #or [0]

df_x = df.groupby(['name']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
df1=df[df['name'].isin(df_x[(df_x['counts']==var1)]['name'].to_list())]  
heatmap1_data = pd.pivot_table(df1, values='rn', 
                     index=['name'],
                     columns='date')
plt.figure(figsize = (15,15))
sns.heatmap(heatmap1_data, center=None, cmap="RdBu")

# *Korreláció*

In [ ]:
df_x = df.groupby(['name']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
df_y = df_x[(df_x['counts']==var1)]
list_y = df_y['name'].to_list()
list_of_values = list_y
df_y = df[df['name'].isin(list_of_values)]
table = pd.pivot_table(df_y, values="rn", index="date", columns="name")
 
plt.figure(figsize=(25,25))
dataplot=sns.heatmap(table.corr(), annot=True)
plt.show()